### <font color ='lightcoral'> Import Packages

In [2]:
install.packages('aricode')
library(aricode)


  Une version binaire est disponible mais la version du source est plus
  récente:
        binary source needs_compilation
aricode  1.0.2  1.0.3              TRUE

  Binaries will be installed
package 'aricode' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Abdoul Wahab\AppData\Local\Temp\RtmpO0iXxg\downloaded_packages


In [3]:
install.packages('fossil')
library(fossil)

Warning message:
"le package 'fossil' est en cours d'utilisation et ne sera pas installé"


In [4]:
install.packages('sparcl')
library(sparcl)

package 'sparcl' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Abdoul Wahab\AppData\Local\Temp\RtmpO0iXxg\downloaded_packages


### <font color ='lightcoral'> Fonction SFGKM_GetWCSS de SFGKM

In [5]:

SFGKM_Getwmodbcss <- function(x, Cs, ws, vs, G){
  # Update the clusters
  # x : the data matrix, dim : (nrow(x), ncol(x))
  # K : the number of clusters
  # ws : the matrix of weights of the features blocks for each class, dim(K, T)
  # vs : the matrix of weights of the variables of each block for each cluster, dim(K, ncol(x))
  # G = list(c(1:t_1), c(t_1+1, t_2), ..., c(t_(T-1)+1,t_T )) : block of features
  # cs : vector of the clusters
  wmodbcss <- matrix(NA, length(unique(Cs)), ncol(x))
  for(k in unique(Cs)){
    whichers <- (Cs==k)
    if(sum(whichers)>1)
        for(t in seq(length(G))){
            wmodbcss[k, G[[t]]] <- (1/ (length(unique(Cs)) * length(G)))*(apply(scale(x[, G[[t]]], center=TRUE, scale=FALSE)^2 * ws[k,t]*vs[k,G[[t]]], 2, sum)-  length(unique(Cs)) * apply(scale(x[whichers,G[[t]]],center=TRUE, scale=FALSE)^2*ws[k,t]*vs[k,G[[t]]], 2, sum))
        }
  }
  return(list(wmodbcss=colSums(wmodbcss)))

 }

### <font color ='lightcoral'> Fonction UpdateCs_SFGKM de SFGKM

In [6]:
UpdateCs_SFGKM <- function(x, K, ws,vs, G, cs){
  # Update the clusters
  # x : the data matrix, dim : (nrow(x), ncol(x))
  # K : the number of clusters
  # ws : the matrix of weights of the features blocks for each class, dim(K, T)
  # vs : the matrix of weights of the variables of each block for each cluster, dim(K, ncol(x))
  # G = list(c(1:t_1), c(t_1+1, t_2), ..., c(t_(T-1)+1,t_T )) : block of features
  # cs : vector of the clusters
  z = matrix(NA,nrow(x), ncol(x))  # matrix of data weighted by (ws_kt*vs_ktj)/T
  x = as.matrix(x)
  for(t in seq(length(G))){
    for(k in unique(cs)){
      z[cs==k,G[[t]]] <- x[cs==k,G[[t]]]*(ws[k,t]/length(G))*vs[k,G[[t]]] # weight the data by (w_kt*v_ktj)/T
      # z[cs==k,G[[t]]] <- x[cs==k,G[[t]]]*ws[k,t]*vs[k,G[[t]]]
    }
  }

  z = z[, colSums(z) != 0] # Delete null columns

  nrowz = nrow(z)
  mus <- NULL
  if(!is.null(cs)){
   for(k in unique(cs)){
      if(sum(cs==k)>1) mus <- rbind(mus, apply(z[cs==k,],2,mean))
      if(sum(cs==k)==1) mus <- rbind(mus, z[cs==k,])

  }

  if(is.null(mus)){
     km <- kmeans(z, centers=K, nstart=10)
  }
  else {
     distmat <- as.matrix(dist(rbind(z, mus)))[1:nrowz, (nrowz+1):(nrowz+K)]
     nearest <- apply(distmat, 1, which.min)

     if(length(unique(nearest))==K){
        km <- kmeans(z, centers=mus)
     }
     else {
         km <- kmeans(z, centers=K, nstart=10)
    }
  }

  return(km$cluster)
  # out = list(cs = km$cluster, inertia = km$inertia)
  # return(out)

 }
}

### <font color ='lightcoral'> Fonctions soft et l2n

In [7]:
soft <- function(x,d){
  # Sign function
  # return(sign(x)*pmax(0, abs(x)-d))
  return(pmax(0, abs(x)-d))
}

l2n <- function(vec){
  # l2 norm function
  return(sqrt(sum(vec^2)))
}

### <font color ='lightcoral'> Fonction BinarySearch

In [8]:
BinarySearch <- function(argu,sumabs){
  # Compute the delta
  if(l2n(argu)==0 || sum(abs(argu/l2n(argu)))<=sumabs) return(0)
  lam1 <- 0
  lam2 <- max(abs(argu))-1e-3
  iter <- 1
  while(iter<=15 && (lam2-lam1)>(1e-2)){
    su <- soft(argu,(lam1+lam2)/2)
    if(sum(abs(su/l2n(su)))<sumabs){
      lam2 <- (lam1+lam2)/2
    } else {
      lam1 <- (lam1+lam2)/2
    }
    iter <- iter+1
  }
  return((lam1+lam2)/2)
}


### <font color ='lightcoral'> Fonction Updatevs de SFGKM

In [9]:
Updatevs_SFGKM <- function(x, Cs,ws,Vs,G, l1bound){
  # Update the feature weights
  # x : the data matrix, dim : (nrow(x), ncol(x))
  # G = list(c(1:t_1), c(t_1+1, t_2), ..., c(t_(T-1)+1,t_T )) : block of features
  # Cs : vector of the features
  # ws : the matrix of the block weights at the previous iteration
  # l1bound : selection parameter s
  lam = matrix(NA,length(unique(Cs)),length(G)) # Matrix of delta_tk
  vs = matrix(NA, length(unique(Cs)),ncol(x))   # Matrix of the weights of the features of each block for each cluster, dim(K, ncol(x))
  a = matrix(NA, length(unique(Cs)),ncol(x))
  for( k in unique(Cs)){
     for( t in seq(length(G))){

         WCSS.perfeature_t_per_cluster = apply(scale(x[Cs==k,G[[t]]],center=TRUE, scale=FALSE)^2, 2, sum) # within-cluster sum of squares for a given group
         tss.perfeature_t <- apply(scale(x[ ,G[[t]]],center=TRUE, scale=FALSE)^2, 2, sum) # total sum of squares for a given group
         a[k,G[[t]]] = ws[k,t]*Vs[k,G[[t]]]*(-WCSS.perfeature_t_per_cluster + tss.perfeature_t) # compute a_tk
         lam[k,t] <- BinarySearch(a[k,G[[t]]], l1bound) # Compute delta_tk
         vs.unscaled <- soft(a[k,G[[t]]],lam[k,t]) # Compute vs_tk
         vs[k,G[[t]]] = vs.unscaled/(l2n(vs.unscaled ) + 0.0000000000001) # We add 0.0000000000001 to avoid having a zero denominator
       }
  }
  return(vs)

}

### <font color ='lightcoral'> Fonction Udpdate_ws de SFGKM

In [10]:
Updatews_SFGKM <- function(x, Cs, G,vs,Ws, l1bound_){
  # Update block features weights
  # x : the data matrix, dim : (nrow(x), ncol(x))
  # Cs : vector of the clusters
  # G = list(c(1:t_1), c(t_1+1, t_2), ..., c(t_(T-1)+1,t_T )) : block of features
  # vs : the matrix of the weights of the features of each block for each class from the previous iteration, dim(K, ncol(x))
  # l1bound_ : selection parameter S
  lam_ = matrix(NA, length(unique(Cs)), length(G)) # Matrix of delta_k
  ws = matrix(NA, length(unique(Cs)), length(G)) # Matrix of weights of feature blocks
  b = matrix(NA, length(unique(Cs)), length(G))
  # ws_k = matrix(NA, length(unique(Cs)), length(G)) # Matrix of ws_k
for( k in unique(Cs)){
      for(t in seq(length(G))){
        WCSS.perfeature_t_per_cluster = apply(scale(x[Cs==k,G[[t]]],center=TRUE, scale=FALSE)^2, 2, sum)
        tss.perfeature_t <- apply(scale(x[ ,G[[t]]],center=TRUE, scale=FALSE)^2, 2, sum) # total sum of squares for a given group
        b[k,t] = Ws[k,t]*sum(vs[k,G[[t]]]*(-WCSS.perfeature_t_per_cluster + tss.perfeature_t))
       }
       lam_[k,] <- BinarySearch(b[k,],l1bound_)
       ws.unscaled <- soft(b[k,],lam_[k,])
       ws[k,] = ws.unscaled/l2n(ws.unscaled)

  }
  return(ws)

}

### <font color ='lightcoral'> Fonction SparseFGKM

In [11]:
SparseFGKM <-
function(x, K=NULL, G,  vbounds=NULL, wbounds= NULL, nstart=20, silent=FALSE, maxiter=40, centers=NULL){
  # The Sparse Features Grouping KMeans function
  # x is the data, nxp
  # K is the number of desired clusters
  # G = list(c(1:t_1), c(t_1+1, t_2), ..., c(t_(T-1)+1,t_T )) : features
  # vbounds vector of the parameters s
  # wbounds vector of the parameters S
  set.seed(11)
  if(is.null(K) && is.null(centers)) stop("Must provide either K or centers.")
  if(!is.null(K) && !is.null(centers)){
    if(nrow(centers)!=K) stop("If K and centers both are provided, then nrow(centers) must equal K!!!")
    if(nrow(centers)==K) K <- NULL
  }
  if(!is.null(centers) && ncol(centers)!=ncol(x)) stop("If centers is provided, then ncol(centers) must equal ncol(x).")
  if(is.null(vbounds)) vbounds <- seq(1.1, sqrt(ncol(x)), len=20)
  if(is.null(wbounds)) wbounds <- seq(0.5, sqrt(length(G)), len=20)
  if(min(vbounds)<=1 | min(wbounds) <= 0.5) stop("wbounds or vboounds should be greater than 1")
  vbounds <- c(vbounds) # In case vbounds is a single number, turn it into a vector
  wbounds <- c(wbounds) # In case wbounds is a single number, turn it into a vector
  out <- list()
  if(!is.null(K)) Cs <- kmeans(x, centers=K, nstart=nstart)$cluster # Initialize Clusters
  if(is.null(K)) Cs <- kmeans(x, centers=centers)$cluster
  for(i in 1:length(vbounds)){
    if(length(vbounds)>1 && !silent) cat(i,fill=FALSE)
       ws = matrix(NA, length(unique(Cs)), length(G)) # Feature blocks weights matrix
       ws_old = matrix(NA, length(unique(Cs)), length(G)) # Feature blocks weights matrix at the prior iteration
       vs = matrix(NA, length(unique(Cs)), ncol(x))  # Feature weights matrix
       vs_old = matrix(NA, length(unique(Cs)), ncol(x)) # Feature weights matrix at the prior iteration
    # Initialize the weights
    for(k in unique(Cs)){
        for(t in seq(length(G))){
          ws[k,t] <- 1/sqrt(length(G)) # Initialize the feature block weights by 1/sqrt(length(G))
          vs[k,G[[t]]] <- rep(1/sqrt(ncol(x)),length(G[[t]])) # Initialize the feature weights by 1/sqrt(length(G))
          ws_old[k,t] <- rnorm(1)
          vs_old[k,G[[t]]] <- rnorm(length(G[[t]]))

           }
    }
    niter <- c(rep(1,length(unique(Cs))))


   for(k in unique(Cs)){

    while((sum(abs(vs[k,]-vs_old[k,]))/sum(abs(vs_old[k,]))) > 1e-4 && (sum(abs(ws[k,]-ws_old[k,]))/sum(abs(ws_old[k,]))) > 1e-4 && niter[k] < maxiter){
      niter[k] <- niter[k]+1
      for(t in seq(length(G))){
         ws_old[k,t] <- ws[k,t]
         vs_old[k,G[[t]]] <- vs[k,G[[t]]]

         if(!is.null(K)){
            if(niter[k]>1) Cs <- UpdateCs_SFGKM(x, K, ws,vs, G, Cs) # if niter[k,t]=1, no need to update!!
         } else {
           if(niter[k]>1) Cs <- UpdateCs_SFGKM(x, nrow(centers), ws,vs, G, Cs) # if niter[k,t]=1, no need to update!!
         }


         Up_vs = Updatevs_SFGKM(x, Cs,ws,vs,G, vbounds[i])
         Up_ws = Updatews_SFGKM(x, Cs, G,vs,ws, wbounds[i])
         vs[k,G[[t]]] <- Up_vs[k,G[[t]]]
         ws[k,t] <- Up_ws[k,t]

      }

      }

    }
    out[[i]] <- list(ws=ws, vs = vs, Cs=Cs, vbounds=vbounds[i], wbounds = wbounds[i])

  }
  return(out)
}

### <font color ='lightcoral'> Experiments

#### <font color ='bisque'> MNIST-4

In [ ]:
# Importer les données
MNIST_4 = scale(as.matrix(read.csv('MNIST-4.csv')))
clusters_MNIST_4 = rep(1:4, each = 1000)
groups = list(
  # ISO (Isomap) : 30 features
  c(1:30),
  # LDA (Linear Discriminant Analysis) : 9 features
  c(31:39),
  # NPE (Neighborhood Preserving Embedding) : 30 features
  c(40:69)
)

In [ ]:
search_params <- function(data_matrix, n_clusters, cluster_labels, groups) {
  vbounds = exp(seq(log(3.2), log(sqrt(ncol(data_matrix)) * 0.9), length.out = 20))
  wbounds = exp(seq(log(1.3), log(sqrt(length(groups)) * 0.9), length.out = 20))
  for (s in vbounds) {
    for (S in wbounds) {
      sfgkm = SparseFGKM(data_matrix, n_clusters, groups, vbounds = s, maxiter = 40, wbounds = S)
      print(sfgkm)
      print(NMI(sfgkm[[1]]$Cs, cluster_labels))
    }
  }
}

In [ ]:
# rechercher les paramètres
search_params(MNIST_4, 4, clusters_MNIST_4, groups)

#### <font color ='bisque'> ProteinFold Data

In [ ]:
ProteinFold_data = scale(as.matrix(read.csv('ProteinFold_data.csv')))
groups1 = list(c(1:27),c(28:54),c(55:81), c(82:108), c(109:135), c(136:162), c(163:189),
  c(190:216),c(217:243),c(244:270), c(271:297), c(298:324))

In [ ]:
head(ProteinFold_data)

view1_0,view1_1,view1_2,view1_3,view1_4,view1_5,view1_6,view1_7,view1_8,view1_9,⋯,view12_17,view12_18,view12_19,view12_20,view12_21,view12_22,view12_23,view12_24,view12_25,view12_26
0.133635286,-0.165940523,0.55640222,-0.37056322,-0.002192521,-0.2445779,0.6166686,0.385529576,-0.51023790,-0.1803035,⋯,1.6530528,0.18623603,0.1687318,0.3015749,-0.01380271,0.57065276,0.29205480,0.72652623,0.20037652,-1.0146591
0.138929378,-0.330974635,0.33666451,0.04671788,-0.183483232,-0.1254099,0.5175544,0.174742711,-0.09659876,0.1529985,⋯,1.5797544,0.46921810,0.6645337,0.1331149,-0.36216332,0.42027333,-0.08569233,-0.03918972,0.04430781,-0.3641095
0.122128327,-0.145647119,-0.23870916,-0.16067528,-0.310133706,0.3048196,1.7818795,-0.070892690,0.16184735,-0.5465455,⋯,0.8463887,-0.01020683,-0.7099481,0.6884352,-0.24815072,-0.01394237,-0.26758880,-0.07213454,0.53717720,-0.2839413
-0.006330186,-0.006075196,-0.49155534,0.13088969,0.027911680,0.8791586,2.7511026,-0.618379201,0.39694468,-0.7095145,⋯,1.3179420,0.32424455,0.7627851,-0.2817482,-0.69514141,-0.22755132,-0.19468385,-0.25439835,-1.40878739,0.2446004
-0.807594755,-0.294584212,0.01366439,-0.26947350,-0.454233872,0.2059881,0.3048732,-0.001729448,0.03432918,-0.4585487,⋯,1.3229042,0.60944880,-0.2276186,0.6362417,-0.82222900,0.16640937,-0.26220372,-0.02777570,0.23724528,-0.9682607
0.060991021,-0.207569592,0.07772336,-0.18282238,-0.276377335,-0.2300734,0.3160704,0.105701834,-0.33007435,-0.0599309,⋯,1.0533250,-0.19645297,0.5438427,0.2427309,0.05458503,-0.33259642,0.13823601,0.20916240,-0.53168712,0.1931307


In [ ]:
install.packages("cluster")
library(cluster)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


Attaching package: ‘cluster’


The following object is masked from ‘package:maps’:

    votes.repub




In [ ]:
search_params_2 <- function(data_matrix, n_clusters, groups) {

  vbounds = exp(seq(log(3.2), log(sqrt(ncol(data_matrix)) * 0.9), length.out = 20))
  wbounds = exp(seq(log(1.3), log(sqrt(length(groups)) * 0.9), length.out = 20))
  
  for (s in vbounds) {
    for (S in wbounds) {
      # 1. Exécution de SparseFGKM
      sfgkm = SparseFGKM(data_matrix, n_clusters, groups, vbounds = s, maxiter = 40, wbounds = S)
      
      # 2. Récupérer les affectations de cluster et calculer la matrice de distance
      clusters = sfgkm[[1]]$Cs
      dist_matrix = dist(data_matrix)
      
      # 3. Calcul de la silhouette
      silhouette_result = silhouette(clusters, dist_matrix)
      
      # 4. Calculer la moyenne des largeurs de silhouette pour la métrique globale
      mean_silhouette = mean(silhouette_result[, "sil_width"])
      
      # 5. Affichage des résultats
      print(sfgkm)
      print(paste("Silhouette Value:", mean_silhouette))
    }
  }
}

In [ ]:
# recherche des paramètres 
search_params_2(ProteinFold_data, 27, groups1)

#### <font color ='bisque'> MSRC-v5 Data

In [ ]:
MSRC_data = scale(as.matrix(read.csv('MSRC-v5.csv')))
groups2 = list(
  # CM (Color Moments) : 24 features
  c(1:24),
  # HOG (Histogram of Oriented Gradients) : 576 features
  c(25:600),
  # GIST : 512 features
  c(601:1112),
  # LBP (Local Binary Patterns) : 256 features
  c(1113:1368),
  # CENT (Centroid) : 254 features
  c(1369:1622)
)

In [ ]:
# recherche des paramètres
search_params_2(MSRC_data, 7, groups2)